### Import Required Libraries and Set Up Environment Variables

In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [5]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}\
&end_date={end_date}"

# Send the GET request
response = requests.get(query_url)

# Print the response
print(response.json())

{'status': 'OK', 'copyright': 'Copyright (c) 2023 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qua

In [7]:
# Retrieve articles
articles = requests.get(query_url).json()
articles_list = articles["response"]["docs"]

for article in articles_list:
    print(f'A snippet from the article: {article["snippet"]}')
    print('---------------------------')

A snippet from the article: A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.
---------------------------
A snippet from the article: Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.
---------------------------
A snippet from the article: Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.
---------------------------
A snippet from the article: Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.
---------------------------
A snippet from the article: A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.
---------------------------
A snippet from the article: A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock m

In [ ]:
# Create an empty list to store the reviews
articles_list_df = []

# loop through pages 0-19
for page in range (0, 20):
    query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}\
&end_date={end_date}"
    
    # create query with a page number
    query_url = f"{query_url}&page={str(page)}"
    articles = requests.get(query_url).json()
    
    # API results show 10 articles at a time
    results_per_page = 10
    page = 1

    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(url)
    #retrieve the JSON data
    data = response.json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
reviews_list = []
        # loop through the reviews["response"]["docs"] and append each review to the list
while True:
    
        params = {
            "page": page,
            "per_page": results_per_page
        }
        # Print the page that was just retrieved
print(params)

        # Print the page number that had no results then break from the loop
if len(data) == 0:
        print("No results found on page", page)
        break
        
    # Make the GET request with the query parameters
import requests

page = 1
has_results = True

while has_results:
    # Make the GET request with the query parameters
    response = requests.get(url, params={'page': page})
    
    # Retrieve the JSON data
    data = response.json()
    
    # Check if the page has results
if len(data) == 0:
        print("No results found on page", page)
        break
    
    # Process the data here...
    
    # Increment the page number
page += 1
    
    # Retrieve the JSON data from the response
data = response.json()
    
    # Check if there are no more results
if len(data) == 0:
        break
    
    # Loop through the reviews and append each review to the reviews_list
for review in data:
    reviews_list.append(review)
    
    # Increment the page number
    page += 1

# Print the reviews_list
    for review in reviews_list:
        print(review)

In [ ]:
# Preview the first 5 results in JSON format
preview_data = reviews_list[:5]

# Use json.dumps with argument indent=4 to format data

formatted_data = json.dumps(preview_data, indent=4)

print(formatted_data)

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
formatted_data = pd.DataFrame(json_normalize(formatted_data))

# Print the DataFrame
print(formatted_data)

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
formatted_data['title'] = formatted_data['headline.main'].str.extract(r'\u2018(.*?) Review\u2019', expand=False)

# Print the updated DataFrame
print(formatted_data)

In [ ]:
formatted_data.head()

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
formatted_data['keywords'] = formatted_data['keywords'].apply(lambda x: ', '.join(x))

# Print the updated DataFrame
print(formatted_data)

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = formatted_data['title'].to_list()

# Print the list of titles
print(title_list)

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results

movie_list = []
# Create a request counter to sleep the requests after a multiple
# of 50 requests
import requests
import time

counter = 0

for page in range(1, total_pages + 1):
    # Make the GET request with the query parameters
    response = requests.get(url, params={'page': page})
    
    # Increment the counter
    counter += 1
    
    # Check if the counter is a multiple of 50
    if counter % 50 == 0:
        print("Sleeping for 5 seconds...")
        time.sleep(5)
    
    # Retrieve the JSON data
    data = response.json()

# Loop through the titles
import requests
import time

counter = 0
for title in title_list:
    # Check if we need to sleep before making a request
    if counter % 50 == 0 and counter != 0:
        print("Sleeping for 5 seconds...")
        time.sleep(5)


    # Add 1 to the request counter
    counter += 1
    
    # Perform a "GET" request for The Movie Database

    request_url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}"
    response = requests.get(request_url)
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
import requests

for title in title_list:
    try:
        # Perform a "GET" request for The Movie Database
        request_url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}"
        response = requests.get(request_url)
        
        # Process the response here...
        
        # Check if a movie is not found
        if response.json()['total_results'] == 0:
            print(f"Movie '{title}' not found.")
        
    except Exception as e:
        print(f"An error occurred while searching for movie '{title}': {str(e)}")

        # Get movie id
for title in title_list:
    try:
        # Perform a "GET" request for The Movie Database
        request_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
        response = requests.get(request_url)
        
        # Process the response here...
        
        # Check if a movie is not found
        if response.json()['total_results'] == 0:
            print(f"Movie '{title}' not found.")
        else:
            # Get the movie ID
            movie_id = response.json()['results'][0]['id']
            
            # Make a request for the full movie details
            full_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
            full_movie_response = requests.get(full_movie_url)
            
            # Extract the genre names into a list
            genres = [genre['name'] for genre in full_movie_response.json()['genres']]
            
            # Print the genre names
            print(f"The genres for movie '{title}' are: {genres}")
        
    except Exception as e:
        print(f"An error occurred while searching for movie '{title}': {str(e)}")

import requests

tmdb_movies_list = []

for title in title_list:
    try:
        # Perform a "GET" request for The Movie Database
        request_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
        response = requests.get(request_url)
        
        # Process the response here...
        
        # Check if a movie is not found
        if response.json()['total_results'] == 0:
            print(f"Movie '{title}' not found.")
        else:
            # Get the movie ID
            movie_id = response.json()['results'][0]['id']
            
            # Make a request for the full movie details
            full_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
            full_movie_response = requests.get(full_movie_url)
            
            # Extract the genre names into a list
            genres = [genre['name'] for genre in full_movie_response.json()['genres']]
            
            # Extract the spoken_languages' English name into a list
            spoken_languages = [language['english_name'] for language in full_movie_response.json()['spoken_languages']]
            
            # Extract the production_countries' name into a list
            production_countries = [country['name'] for country in full_movie_response.json()['production_countries']]
            
            # Create a dictionary with the relevant data
            movie_data = {
                'title': title,
                'genres': genres,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries
            }
            
            # Append the movie data to the tmdb_movies_list list
            tmdb_movies_list.append(movie_data)
            
            # Print the title that was found
            print(f"Movie '{title}' found.")
        
    except Exception as e:
        print(f"An error occurred while searching for movie '{title}': {str(e)}")



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

for title in title_list[:5]:
    try:
        # Perform a "GET" request for The Movie Database
        request_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
        response = requests.get(request_url)
        
        # Process the response here...
        
        # Check if a movie is not found
        if response.json()['total_results'] == 0:
            print(f"Movie '{title}' not found.")
        else:
            # Get the movie ID
            movie_id = response.json()['results'][0]['id']
            
            # Make a request for the full movie details
            full_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
            full_movie_response = requests.get(full_movie_url)
            
            # Extract the genre names into a list
            genres = [genre['name'] for genre in full_movie_response.json()['genres']]
            
            # Extract the spoken_languages' English name into a list
            spoken_languages = [language['english_name'] for language in full_movie_response.json()['spoken_languages']]
            
            # Extract the production_countries' name into a list
            production_countries = [country['name'] for country in full_movie_response.json()['production_countries']]
            
            # Create a dictionary with the relevant data
            movie_data = {
                'title': title,
                'genres': genres,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries
            }
            
            # Append the movie data to the tmdb_movies_list list
            tmdb_movies_list.append(movie_data)
            
            # Print the title that was found
            print(f"Movie '{title}' found.")
        
    except Exception as e:
        print(f"An error occurred while searching for movie '{title}': {str(e)}")

# Preview the first 5 results in JSON format
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
df = pd.DataFrame(tmdb_movies_list)

# Print the DataFrame
print(df)

# Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
tmdb_movies_list = []

# ... Code to populate tmdb_movies_list ...

# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

# Merge the New York Times reviews DataFrame and the TMDB DataFrame on the "title" column
merged_df = pd.merge(nyt_reviews_df, tmdb_df, on="title")

# Print the merged DataFrame
print(merged_df)

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['Column1', 'Column2', 'Column3']

# Create a list of characters to remove
characters_to_remove = ['[', ']', '"']

# Loop through the list of columns to fix
for column in columns_to_fix:
    
     # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
   
  for char in characters_to_remove:
        # Use str.replace() method to remove the character
        merged_df[column] = merged_df[column].str.replace(char, '')

    # Loop through characters to remove
    for char in characters_to_remove:
        # Use str.replace() method to remove the character
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df)

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop("byline.person", axis=1)

# Display the updated DataFrame
print(merged_df)

In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicated()

merged_df = merged_df.reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv(data.csv, index=False)

In [ ]:
# Hi Matlree Maniar - I really ap